In [69]:
import pandas as pd
from sqlalchemy import create_engine
import pymongo

### Store CSV into DataFrame

In [70]:
# DATA FILES
data_shootings_path = "data/data_shootings.csv"
data_shootings_locations_path = "data/data_shootings_locations.csv"

# READ
shootings_df = pd.read_csv(data_shootings_path)
locations_df = pd.read_csv(data_shootings_locations_path)

In [71]:
shootings_df.head(3)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False


In [72]:
locations_df.head(3)

,city,lon,lat
0,Los Angeles CA,-118.244476,34.054935
1,Phoenix AZ,-112.077346,33.448587
2,Houston TX,-95.367697,29.758938


In [74]:
# CLEAN LOCATIONS DF.

# REMOVE STATE FROM CITY COLUMN.
locations_df_2 = locations_df.city.str.rsplit(" ", 1, expand=True).rename(lambda x: f'col{x + 1}', axis=1)
locations_df_2

# RENAME COLUMNS
locations_df_2  = locations_df_2.rename(columns={'col1':'City'})
locations_df_2 = locations_df_2.rename(columns={'col2':'State'})
locations_df_2

,City,State
0,Los Angeles,CA
1,Phoenix,AZ
2,Houston,TX
3,Chicago,IL
4,Las Vegas,NV
...,...,...
1999,Gainesville,GA
2000,Martins Ferry,OH
2001,Soddy-Daisy,TN
2002,Kearny,AZ


In [75]:
# MERGE BOTH DFs
merged_loc_df = locations_df_2.join(other=locations_df,how='left')

#DROP EXTRA CITY COLUMN
merged_loc_df = merged_loc_df.drop('city', 1)
merged_loc_df

,City,State,lon,lat
0,Los Angeles,CA,-118.244476,34.054935
1,Phoenix,AZ,-112.077346,33.448587
2,Houston,TX,-95.367697,29.758938
3,Chicago,IL,-87.624421,41.875555
4,Las Vegas,NV,-115.149225,36.166286
...,...,...,...,...
1999,Gainesville,GA,-83.824066,34.297879
2000,Martins Ferry,OH,-80.724526,40.095906
2001,Soddy-Daisy,TN,-85.190790,35.235903
2002,Kearny,AZ,-110.910666,33.057009


In [76]:
# MATCH CUTY COLUMN NAME CASE FOR THE NEXT STEP
merged_loc_df  = merged_loc_df.rename(columns={'City':'city'})
merged_loc_df

,city,State,lon,lat
0,Los Angeles,CA,-118.244476,34.054935
1,Phoenix,AZ,-112.077346,33.448587
2,Houston,TX,-95.367697,29.758938
3,Chicago,IL,-87.624421,41.875555
4,Las Vegas,NV,-115.149225,36.166286
...,...,...,...,...
1999,Gainesville,GA,-83.824066,34.297879
2000,Martins Ferry,OH,-80.724526,40.095906
2001,Soddy-Daisy,TN,-85.190790,35.235903
2002,Kearny,AZ,-110.910666,33.057009


In [77]:
# MERGE SHOOTINGS DF WITH LOCATIONS DF
merge_df = pd.merge(merged_loc_df, shootings_df, on='city', how='outer')
merge_df.head(3)

,city,State,lon,lat,id,name,date,manner_of_death,armed,age,gender,race,state,signs_of_mental_illness,threat_level,flee,body_camera
0,Los Angeles,CA,-118.244476,34.054935,75.0,Pablo Meza,2015-01-17,shot,gun,24.0,M,H,CA,False,attack,Not fleeing,False
1,Los Angeles,CA,-118.244476,34.054935,194.0,Charly Leundeu Keunang,2015-03-01,shot and Tasered,unarmed,43.0,M,B,CA,True,attack,Not fleeing,True
2,Los Angeles,CA,-118.244476,34.054935,796.0,Aaron Valdez,2015-03-11,shot,NaN,25.0,M,H,CA,False,other,Not fleeing,False


In [78]:
# DROP EXTRA STATE COLUMN
merge_df = merge_df.drop('state' , 1)
merge_df.head(3)

,city,State,lon,lat,id,name,date,manner_of_death,armed,age,gender,race,signs_of_mental_illness,threat_level,flee,body_camera
0,Los Angeles,CA,-118.244476,34.054935,75.0,Pablo Meza,2015-01-17,shot,gun,24.0,M,H,False,attack,Not fleeing,False
1,Los Angeles,CA,-118.244476,34.054935,194.0,Charly Leundeu Keunang,2015-03-01,shot and Tasered,unarmed,43.0,M,B,True,attack,Not fleeing,True
2,Los Angeles,CA,-118.244476,34.054935,796.0,Aaron Valdez,2015-03-11,shot,NaN,25.0,M,H,False,other,Not fleeing,False


In [79]:
merge_df.isnull().sum()

city                         0
State                      723
lon                        723
lat                        723
id                           2
name                         2
date                         2
manner_of_death              2
armed                      312
age                        308
gender                       4
race                       698
signs_of_mental_illness      2
threat_level                 2
flee                       321
body_camera                  2
dtype: int64

In [80]:
# DROP NULL VALUES AND UNDESIRED DATA

merge_df = merge_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# CHANGE STATE TO LOWER CASE
merge_df  = merge_df.rename(columns={'State':'state'})
merge_df

merge_df.head(3)

,city,state,lon,lat,id,name,date,manner_of_death,armed,age,gender,race,signs_of_mental_illness,threat_level,flee,body_camera
0,Los Angeles,CA,-118.244476,34.054935,75.0,Pablo Meza,2015-01-17,shot,gun,24.0,M,H,False,attack,Not fleeing,False
1,Los Angeles,CA,-118.244476,34.054935,194.0,Charly Leundeu Keunang,2015-03-01,shot and Tasered,unarmed,43.0,M,B,True,attack,Not fleeing,True
3,Los Angeles,CA,-118.244476,34.054935,331.0,Roberto Rodriguez,2015-04-08,shot,gun,39.0,M,H,False,attack,Foot,False


In [81]:
rds_connection_string = "postgres:postgres@localhost:5432/police_shootings_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [82]:
engine.table_names()

['police_shootings_db', 'police_shootings']

### Use pandas to load csv converted DataFrame into database

In [83]:
merge_df.to_sql(name='police_shootings', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [84]:
pd.read_sql_query('select * from police_shootings', con=engine).head()

,city,state,lon,lat,id,name,date,manner_of_death,armed,age,gender,race,signs_of_mental_illness,threat_level,flee,body_camera
0,Los Angeles,CA,-118.244476,34.054935,75.0,Pablo Meza,2015-01-17,shot,gun,24.0,M,H,False,attack,Not fleeing,False
1,Los Angeles,CA,-118.244476,34.054935,194.0,Charly Leundeu Keunang,2015-03-01,shot and Tasered,unarmed,43.0,M,B,True,attack,Not fleeing,True
2,Los Angeles,CA,-118.244476,34.054935,331.0,Roberto Rodriguez,2015-04-08,shot,gun,39.0,M,H,False,attack,Foot,False
3,Los Angeles,CA,-118.244476,34.054935,495.0,Luis Martinez,2015-04-21,shot,knife,35.0,M,H,True,other,Not fleeing,False
4,Los Angeles,CA,-118.244476,34.054935,619.0,Jason Hendley,2015-07-06,shot,knife,29.0,M,B,False,attack,Not fleeing,False


In [102]:
#CONNECT MONGO DB

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [103]:
#db = client.police_shootings_db
client.police_shootings_db.police_shootings.insert_many(merge_df.to_dict('records'))


In [104]:
police_shootings = db.police_shootings.find()
print(police_shootings)

In [105]:
for shootings in police_shootings:
   print(shootings)

{'_id': ObjectId('5edeff6eea314b243d217469'), 'city': 'Los Angeles', 'state': 'CA', 'lon': -118.244476, 'lat': 34.054935, 'id': 75.0, 'name': 'Pablo Meza', 'date': '2015-01-17', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 24.0, 'gender': 'M', 'race': 'H', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d21746a'), 'city': 'Los Angeles', 'state': 'CA', 'lon': -118.244476, 'lat': 34.054935, 'id': 194.0, 'name': 'Charly Leundeu Keunang', 'date': '2015-03-01', 'manner_of_death': 'shot and Tasered', 'armed': 'unarmed', 'age': 43.0, 'gender': 'M', 'race': 'B', 'signs_of_mental_illness': True, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': True}
{'_id': ObjectId('5edeff6eea314b243d21746b'), 'city': 'Los Angeles', 'state': 'CA', 'lon': -118.244476, 'lat': 34.054935, 'id': 331.0, 'name': 'Roberto Rodriguez', 'date': '2015-04-08', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 39.

{'_id': ObjectId('5edeff6eea314b243d2177cb'), 'city': 'Charlotte', 'state': 'NC', 'lon': -80.8431268, 'lat': 35.2270869, 'id': 1140.0, 'name': 'Germonta Wallace', 'date': '2016-01-03', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 30.0, 'gender': 'M', 'race': 'B', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d2177cc'), 'city': 'Charlotte', 'state': 'NC', 'lon': -80.8431268, 'lat': 35.2270869, 'id': 1147.0, 'name': 'Carlton Antonio Murphy', 'date': '2016-01-05', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 33.0, 'gender': 'M', 'race': 'B', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d2177cd'), 'city': 'Charlotte', 'state': 'NC', 'lon': -80.8431268, 'lat': 35.2270869, 'id': 1474.0, 'name': 'Sylasone Ackhavong', 'date': '2016-04-19', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 41.0, 'ge

{'_id': ObjectId('5edeff6eea314b243d217bb3'), 'city': 'Casper', 'state': 'WY', 'lon': -119.07295, 'lat': 35.349404, 'id': 1642.0, 'name': 'Jeff Hyde', 'date': '2016-06-21', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 50.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': True, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d217bb4'), 'city': 'Casper', 'state': 'WY', 'lon': -119.07295, 'lat': 35.349404, 'id': 3444.0, 'name': 'Douglas Oneyear', 'date': '2018-02-25', 'manner_of_death': 'shot', 'armed': 'sword', 'age': 36.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': False, 'threat_level': 'other', 'flee': 'Foot', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d217bb5'), 'city': 'Casper', 'state': 'WY', 'lon': -119.07295, 'lat': 35.349404, 'id': 3673.0, 'name': 'David P. Wolosin', 'date': '2018-05-06', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 38.0, 'gender': 'M', 'race': 'W', 'signs_of_men

{'_id': ObjectId('5edeff6eea314b243d217f9a'), 'city': 'Salisbury', 'state': 'NC', 'lon': 24.776729, 'lat': -28.7464779, 'id': 5732.0, 'name': 'Rick Howell', 'date': '2020-04-08', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 59.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': True, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d217f9b'), 'city': 'Salisbury', 'state': 'MD', 'lon': -75.60088809999999, 'lat': 38.3662114, 'id': 197.0, 'name': 'Fednel Rhinvil', 'date': '2015-03-03', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 25.0, 'gender': 'M', 'race': 'B', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d217f9c'), 'city': 'Salisbury', 'state': 'MD', 'lon': -75.60088809999999, 'lat': 38.3662114, 'id': 2011.0, 'name': 'Ferguson Laurent', 'date': '2016-11-03', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 23.0, 'gender

{'_id': ObjectId('5edeff6eea314b243d21839e'), 'city': 'Sparta', 'state': 'WI', 'lon': -90.8129118, 'lat': 43.9441328, 'id': 1462.0, 'name': 'Koltlee Whitson', 'date': '2016-04-13', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 26.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Car', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d21839f'), 'city': 'Sparta', 'state': 'WI', 'lon': -90.8129118, 'lat': 43.9441328, 'id': 3280.0, 'name': 'Skyler Burnette', 'date': '2018-01-06', 'manner_of_death': 'shot', 'armed': 'knife', 'age': 21.0, 'gender': 'M', 'race': 'N', 'signs_of_mental_illness': False, 'threat_level': 'other', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d2183a0'), 'city': 'Sparta', 'state': 'WI', 'lon': -90.8129118, 'lat': 43.9441328, 'id': 3613.0, 'name': 'Chad Eric Montgomery', 'date': '2018-04-11', 'manner_of_death': 'shot', 'armed': 'machete', 'age': 39.0, 'gender': 'M', 'race'

{'_id': ObjectId('5edeff6eea314b243d218769'), 'city': 'Peterson', 'state': 'MN', 'lon': -111.9434096, 'lat': 33.3783859, 'id': 1984.0, 'name': 'Kristofer Daniel Youngquist', 'date': '2016-10-23', 'manner_of_death': 'shot', 'armed': 'toy weapon', 'age': 45.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d21876a'), 'city': 'Volant', 'state': 'PA', 'lon': -80.25895179999999, 'lat': 41.11367070000001, 'id': 3582.0, 'name': 'William Frazier', 'date': '2018-04-05', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 40.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': False}
{'_id': ObjectId('5edeff6eea314b243d21876b'), 'city': 'Sharon', 'state': 'PA', 'lon': -80.4934035, 'lat': 41.2331116, 'id': 2197.0, 'name': 'Sean Ryan Hake', 'date': '2017-01-06', 'manner_of_death': 'shot', 'armed': 'knife',

{'_id': ObjectId('5edeff89ea314b243d218b52'), 'city': 'Louisville', 'state': 'KY', 'lon': -85.759407, 'lat': 38.254237599999996, 'id': 5630.0, 'name': 'Justin Griggs', 'date': '2020-03-14', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 30.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Foot', 'body_camera': True}
{'_id': ObjectId('5edeff89ea314b243d218b53'), 'city': 'Louisville', 'state': 'KY', 'lon': -85.759407, 'lat': 38.254237599999996, 'id': 5699.0, 'name': 'Jessie Stringfield', 'date': '2020-03-30', 'manner_of_death': 'shot', 'armed': 'gun', 'age': 44.0, 'gender': 'M', 'race': 'W', 'signs_of_mental_illness': False, 'threat_level': 'attack', 'flee': 'Not fleeing', 'body_camera': True}
{'_id': ObjectId('5edeff89ea314b243d218b54'), 'city': 'Louisville', 'state': 'NE', 'lon': -85.57618606, 'lat': 38.31059285, 'id': 548.0, 'name': 'Deng Manyoun', 'date': '2015-06-13', 'manner_of_death': 'shot', 'armed': 'flagpole', 'age': 35.0, 'g

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [110]:
merge_df.to_json("police_shootings.json")

In [111]:
merge_df.to_csv("police_shootings.csv")